#### model(openai-gpt)

In [1]:
from langchain_openai import ChatOpenAI
import os

with open('../api-key', 'r') as file:
  api_key = file.read().replace('\n', '')
os.environ["OPENAI_API_KEY"] = api_key

llm = ChatOpenAI(model="gpt-4o-mini", temperature = 0)

#### travily

In [2]:
from tavily import TavilyClient
with open('../tavily-api-key', 'r') as file:
  api_key = file.read().replace('\n', '')
tavily = TavilyClient(api_key=api_key)

#### Rag

In [3]:
### Index

from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Add to vectorDB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding = OpenAIEmbeddings(model="text-embedding-3-small")
)
retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


#### rag_chain

In [4]:
### Generate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system = """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise
    Add a source for the answer (URL, Title, Content).
    """

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "question: {question}\n\n context: {context} "),
    ]
)

# Chain
rag_chain = prompt | llm | StrOutputParser()

#### Relevance Checker

In [5]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

### Relevance Checker

system = """You are a grader assessing relevance
  of a retrieved document to a user question. If the document contains keywords related to the user question,
  grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
  Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
  Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
  """

prompt = ChatPromptTemplate.from_messages(
  [
    ("system", system),
    ("human", "question: {question}\n\n document: {document} "),
  ]
)

relevance_checker = prompt | llm | JsonOutputParser()

#### Hallucination Grader

In [6]:
### Hallucination Grader

system = """You are a grader assessing whether
    an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a
    single key 'score' and no preamble or explanation."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "documents: {documents}\n\n answer: {generation} "),
    ]
)

hallucination_grader = prompt | llm | JsonOutputParser()

#### Answer Grader

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)


# Prompt
system = """You are a grader assessing whether an
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "question: {question}\n\n answer: {generation} "),
    ]
)

answer_grader = prompt | llm | JsonOutputParser()

In [8]:
from pprint import pprint
from typing import List

from langchain_core.documents import Document
from typing_extensions import TypedDict

from langgraph.graph import END, StateGraph

### State


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        generation_tries: number of generation tries
        web_search: whether to add search
        web_search_tries: number of web search tries
        documents: list of documents
    """

    question: str
    generation: str
    generation_tries: int = 0
    web_search: str
    web_search_tries: int = 0
    answer_available: str
    documents: List[str]


### Nodes

def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    print(question)
    print(documents)
    return {"documents": documents, "question": question}


def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    if state.get("generation_tries") is None:
        state["generation_tries"] = 0
    state["generation_tries"] += 1

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}


def relevance_check(state):
    """
    Check relevance of documents to question
    If all documents are not relevant, we will run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web search results to documents
    """
    print("---RELEVANCE CHECK---")
    question = state["question"]
    documents = state["documents"]

    web_search = "Yes"
    for d in documents:
        score = relevance_checker.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score["score"]
        if grade == "yes":
            print("---DOCUMENT IS RELEVANT---")
            web_search = "No"
            break

    if web_search == "Yes":
        if state.get("web_search_tries") is None:
            state['web_search_tries'] = 0
        state['web_search_tries'] += 1
        if state['web_search_tries'] == 1:
            return {"documents": documents, "question": question, "web_search": web_search}
        else:
            # We have tried web search once, print  "failed: not relevant" and end the conversation
            print("failed: not relevant")
            return END
    
    return {"documents": documents, "question": question, "web_search": web_search}


def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    print(state)
    question = state["question"]
    documents = None
    if "documents" in state:
      documents = state["documents"]

    # Web search
    docs = tavily.search(query=question)['results']


    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question}

def hallucination_check(state):
    """
    Check hallucination of generated answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---HALLUCINATION CHECK---")
    generation = state["generation"]
    documents = state["documents"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score["score"]
    if grade == "yes":
        print("---GENERATION IS GROUNDED IN DOCUMENTS---")
        return {"documents": documents, "generation": generation, "answer_available": "Yes"}
    else:
        print("---GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return {"documents": documents, "generation": generation, "answer_available": "No"}

### Edges

def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    state["question"]
    web_search = state["web_search"]
    state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---"
        )
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"
    
def decide_to_regenerate(state):
    """
    Determines whether to regenerate an answer

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    answer_available = state["answer_available"]

    if answer_available == "Yes":
        # answer is grounded in documents
        # answer to user with conference
        print("---DECISION: ANSWER AVAILABLE---")
        return "end"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: RE-GENERATE---")
        if state["generation_tries"] > 1:
            print("---FAILED: HALLUCINATION---")
            return "end"
        return "generate"


workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search)  # web search
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("relevance_check", relevance_check)  # relevance check
workflow.add_node("hallucination_check", hallucination_check)  # hallucination check
workflow.add_node("generate", generate)  # generatae

#### graph flow 작성

In [9]:
# Build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "relevance_check")
workflow.add_conditional_edges(
    "relevance_check",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "relevance_check")
workflow.add_edge("generate", "hallucination_check")
workflow.add_conditional_edges(
    "hallucination_check",
    decide_to_regenerate,
    {
        "generate": "generate",
        "end": END,
    },
)

In [10]:
# Compile 
app = workflow.compile()

In [11]:
# Test 1 - RagChain

inputs = {"question": "What is prompt?"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---RETRIEVE---
What is prompt?
[Document(metadata={'description': 'Prompt Engineering, also known as In-Context Prompting, refers to methods for how to communicate with LLM to steer its behavior for desired outcomes without updating the model weights. It is an empirical science and the effect of prompt engineering methods can vary a lot among models, thus requiring heavy experimentation and heuristics.\nThis post only focuses on prompt engineering for autoregressive language models, so nothing with Cloze tests, image generation or multimodality models.', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/', 'title': "Prompt Engineering | Lil'Log"}, page_content='Prompt Engineering, also known as In-Context Prompting, refers to methods for how to communicate with LLM to steer its behavior for desired outcomes without updating the model weights. It is an empirical science and the effect of prompt engineering methods can vary a lot among models, 

In [12]:
# Test 2 - Web Search

inputs = {"question": "Tell me the weather in Seoul"}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

---RETRIEVE---
Tell me the weather in Seoul
[Document(metadata={'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview In a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:', 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log"}, page_content='"content": "You will get instructions for code to write.\\nYou will write a very long answer. Make sure that every detail of the architecture is, in the end, implemented as code.\\nMake sure that every detail of the architecture is, in the end, implemented as code.\\n\\nThin